## **Logika Metod Żądania FastAPI**

### **Prosta Klasa Danych**

In [50]:
from dataclasses import dataclass


@dataclass(kw_only=True)
class Book:
    """Represents a book with a title, author, and category."""

    title: str
    author: str
    category: str


BOOKS = [
    Book(title="The Great Gatsby", author="F. Scott Fitzgerald", category="Fiction"),
    Book(title="The Da Vinci Code", author="Dan Brown", category="Thriller"),
    Book(title="The Catcher in the Rye", author="J.D. Salinger", category="Fiction"),
    Book(title="The Alchemist", author="Paulo Coelho", category="Fantasy"),
    Book(title="The Hobbit", author="J.R.R. Tolkien", category="Fantasy"),
]

Prosta klasa danych `Book` posłuży do celów testowych w trakcie korzystania z FastAPI. W odniesieniu do listy książek będziemy wykonywać tzw. operacje **CRUD**:
- CREATE (C) -> POST
- READ (R) -> GET
- UPDATE (U) -> PUT
- DELETE (D) -> DELETE
  
Krótko mówiąc poprzez API będziemy w stanie utworzyć nowe książki, odczytać informacje o książkach, zaktualizować książki oraz ewentualnie usunąć je z danej listy.

### **Prosta Aplikacja FastAPI**

In [51]:
from fastapi import Body, FastAPI

app = FastAPI()


@app.get("/books")
async def first_api_endpoint() -> list[Book]:
    return BOOKS

Funkcja `first_api_endpoint()` to zwykła funkcja Pythona zwracająca listę. Jako, że chcemy mieć możliwość czytania lub wysyłania informacji do klienta, należy dodać tzw. endpoint, tak aby klient mógł konsumować daną funkcję. Robimy to poprzez dodanie dekoratora `@app.get("/books")`. Kiedy uruchomimy aplikację i przejdziemy pod adres `localhost:8000/books`, otrzymamy odpowiedź z serwera, którą jest lista książek zwracana w funkcji `first_api_endpoint()`. Tak więc endpoint jest ścieżką, którą definiujemy aby móc wywłować daną funkcję Pythona. Domyślnie podczas definiowania endpointa, słowo kluczowe `async` nie jest potrzebne, gdyż FastAPI zrobi to domyślnie.

### **Uruchamianie Aplikacji FastAPI**

Uruchamianie aplikacji FastAPI następuje poprzez uruchomienie serwera `uvicorn`.

```bash
uvicorn <FILE>:<APP> --reload
```

- uvicorn - serwer internetowy instalowany wraz z FastAPI
- file - jest to plik `.py`, który definiuje aplikację FastAPI
- app - jest to instancja FastAPI
- --reload - przeładowuje aplikację za każdym razem kiedy wprowadzane są jakieś zmiany

Alternatywą jest:

```bash
fastapi run <FILE> --reload
```

### **Parametry Ścieżki**

Parametry ścieżki to parametry żądania, które są dołączane do URL i są zwykle używane do znalezienia informacji na podst. lokalizacji. 

In [52]:
@app.get("/books")
async def get_books() -> list[Book]:
    return BOOKS

W powyższym kodzie ścieżka `/books` jest statyczna i FastAPI nie jest w stanie jej modyfikować. Nie mniej jednak, FastAPI pozwala na definiowanie ścieżek dynamicznych. Osiągamy to poprzez przekazanie parametru w ścieżce. Patrz przykład poniżej.

In [53]:
@app.get("/books/{author}")
async def get_books_by_author(author: str) -> list[Book]:
    return list(book for book in BOOKS if book.author.casefold() == author.casefold())

**Dynamiczny parametr endpointa musi mieć taką samą nazwę jak parametr funkcji dekorowanej.**

### **Parametry Zapytań**

Parametry zapytań to parametry żądań, które zostały dołączone po znaku "?" i zostały zdefiniowane poprzez pary `key=value`. Przykładowo: `localhost:8000/books/?category=fantasy` zwróci wszystkie książki z kategorii fantasy. Patrz przykład poniżej.

In [54]:
@app.get("/books/")
async def get_books_by_query_category(category: str) -> list[Book]:
    return list(book for book in BOOKS if book.category.casefold() == category.casefold())

Parametry zapytań i parametry ścieżki mogą być ze sobą łączone.

### **Metody Żądania POST**

Metoda żądania POST jest używana do tworzenia danych. Metoda POST może mieć dodatkową treść z opcjonalnymi informacjami, których nie mają metody żądania GET.

In [55]:
@app.post("/books/create_book")
async def create(new_book: Book = Body()) -> None:
    BOOKS.append(new_book)

`Body` służy do określenia, że parametr powinien zostać odczytany z treści żądania. Aby wyraźnie to określić, używana jest funkcja `Body()`. `Body()` w tym kontekście służy do poinformowania FastAPI, że parametr `new_book` powinien zostać wyodrębniony z treści JSON przychodzącego żądania POST. Pozwala to FastAPI na przeanalizowanie treści żądania i zweryfikowanie go względem modelu `Book` przed przekazaniem go do funkcji create.

### **Metody Żądania PUT**

Metoda żądania PUT jest używana do aktualizacji danych. Metoda PUT może mieć dodatkową treść z opcjonalnymi informacjami, których nie mają metody żądania GET.

In [56]:
@app.put("/books/update_book")
async def update(updated_book: Book = Body()) -> None:
    for book in BOOKS:
        if book.title.casefold() == updated_book.title.casefold():
            book.author = updated_book.author
            book.category = updated_book.category

Na przykład, zamiast tworzyć nową książkę, przekażemy książkę, która już istnieje, ale będziemy w stanie zmienić kategorię lub autora.

### **Metody Żądania DELETE**

Metoda żądania DELETE jest używana do usuwania danych. 

In [57]:
@app.delete("/books/delete_book/{title}")
async def delete(book_title: str) -> None:
    for book in reversed(BOOKS):
        if book.title.casefold() == book_title.casefold():
            BOOKS.remove(book)